# *ETL Process for Retail Data Warehouse*


#### *Load and preprocess the dataset*

In [1]:
import pandas as pd
import random
import sqlite3
import os

from datetime import datetime
import logging
from glob import glob
import hashlib
from faker import Faker

# *Data Generation: Dimensions*

#### *Generate synthetic Customer Dimension*

In [2]:
random.seed(42)
Faker.seed(42)

*Description:*

*This chunk focuses on loading the raw dataset from a CSV file into a pandas DataFrame, ensuring that special characters in the data are correctly handled by specifying the appropriate encoding. The 'InvoiceDate' column is explicitly converted into a datetime object, which enables more efficient and accurate manipulation of date and time data. To make the dataset appear current for analysis purposes, all invoice dates are shifted forward by 14 years, adjusting the original 2010-2011 timestamps to approximately 2024-2025. This simulated recency of the data can be important for testing or reporting. Finally, the new date range is printed as a sanity check to confirm that the shift was applied correctly.*


In [3]:
# Load dataset CSV into pandas DataFrame.
# Encoding ISO-8859-1 is used to handle special characters.
df = pd.read_csv('../Data/Online_Retail.csv', encoding='ISO-8859-1')

# Convert 'InvoiceDate' column to datetime type for easier date/time operations.
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Shift invoice dates forward by 14 years to simulate current data (2024-2025).
df['InvoiceDate'] = df['InvoiceDate'] + pd.DateOffset(years=14)

# Verify date range after shifting.
print("Date range after shifting:", df['InvoiceDate'].min(), "to", df['InvoiceDate'].max())

Date range after shifting: 2024-12-01 08:26:00 to 2025-12-09 12:50:00


### *Create Time Dimension Table (TimeDim)*
*Description:*

*In this step, a Time Dimension table is constructed, which is fundamental in data warehousing and analytical processing for providing rich temporal context to sales data. The process starts by extracting all unique dates from the transactional data and normalizing them to remove time components, ensuring that each date appears only once. A unique `TimeID` is generated for each date using the YYYYMMDD format, facilitating efficient joins with fact tables. Additional columns are created to break down each date into components such as day, month, quarter, year, and week number — all valuable for time-based grouping and trend analysis. This denormalized structure simplifies querying and reporting over time.*


In [4]:
# 2. Create the Time Dimension (TimeDim) table
# Create empty DataFrame for Time Dimension
time_dim = pd.DataFrame()

# Extract unique dates from the 'InvoiceDate' column (date only, no time)
time_dim['FullDate'] = pd.to_datetime(df['InvoiceDate'].dt.date.unique())

# Generate a unique TimeID for each date in YYYYMMDD integer format
time_dim['TimeID'] = time_dim['FullDate'].dt.strftime('%Y%m%d').astype(int)

# Extract useful date attributes for analysis
time_dim['Day'] = time_dim['FullDate'].dt.day
time_dim['Month'] = time_dim['FullDate'].dt.month
time_dim['Quarter'] = time_dim['FullDate'].dt.quarter
time_dim['Year'] = time_dim['FullDate'].dt.year
time_dim['WeekOfYear'] = time_dim['FullDate'].dt.isocalendar().week

# Reorder columns for clarity
time_dim = time_dim[['TimeID', 'FullDate', 'Day', 'Month', 'Quarter', 'Year', 'WeekOfYear']]

### *Create Customer Dimension Table (CustomerDim)*

*Description:*

*This chunk builds the Customer Dimension table, which profiles unique customers using a combination of actual and synthetic data. The real `CustomerID` and country information are directly extracted to maintain referential integrity. Since personal identifying information like names and cities are not available or desirable to use, synthetic values are generated to enrich the dataset while preserving privacy. Customer names are created by hashing the `CustomerID` to produce consistent yet anonymous identifiers. Cities are generated using the Faker library with locale settings based on the customer’s country, adding realistic geographic diversity. Additionally, plausible gender and age values are randomly assigned within reasonable bounds to simulate demographic attributes. Finally, the earliest invoice date is used as a proxy for the customer’s registration date, providing a temporal reference for customer activity.*


In [5]:
fake = Faker()

def hash_customer_name(cust_id):
    # Generate a synthetic name by hashing the CustomerID
    return hashlib.sha256(str(cust_id).encode()).hexdigest()[:10]

def generate_city_based_on_country(country):
    # Use Faker locale based on country for city name if possible, else default locale
    # Here we simplify: if country is UK use en_GB, else en_US or default
    if country == 'United Kingdom':
        fake_local = Faker('en_GB')
    else:
        fake_local = Faker()
    return fake_local.city()

# Extract unique customers with country
customer_dim = df[['CustomerID', 'Country']].drop_duplicates().copy()

# Create synthetic CustomerName by hashing CustomerID
customer_dim['CustomerName'] = customer_dim['CustomerID'].apply(hash_customer_name)

# Generate synthetic City based on Country
customer_dim['City'] = customer_dim['Country'].apply(generate_city_based_on_country)

# Generate reasonable synthetic Gender and Age
gender_choices = ['Male', 'Female', 'Other']
customer_dim['Gender'] = [random.choice(gender_choices) for _ in range(len(customer_dim))]
customer_dim['Age'] = [random.randint(18, 75) for _ in range(len(customer_dim))]

# Set CustomerSince as earliest InvoiceDate in the dataset
customer_dim['CustomerSince'] = df['InvoiceDate'].min()

# Drop Email column (not required)
# No Email column added here

customer_dim.head()

,CustomerID,Country,CustomerName,City,Gender,Age,CustomerSince
0,17850.0,United Kingdom,54cde5dbb6,North Henrybury,Other,71,2024-12-01 08:26:00
9,13047.0,United Kingdom,86314fa849,East Timothy,Male,69,2024-12-01 08:26:00
26,12583.0,France,dcff63cd99,New Roberttown,Male,74,2024-12-01 08:26:00
46,13748.0,United Kingdom,590354e49f,East Donaldhaven,Other,49,2024-12-01 08:26:00
65,15100.0,United Kingdom,58ec7997a6,New Joeside,Female,52,2024-12-01 08:26:00


### *Create Store Dimension Table (StoreDim)*


*Description:*

*This step constructs the Store Dimension table, which represents the stores or sales locations for transactions. Since the dataset primarily references countries rather than specific store locations, each unique country is treated as a distinct store. Unique numeric `StoreID`s are assigned for efficient foreign key references. Store names are generated to include the country name for clarity and uniqueness. The sales channel is hardcoded as "Online," reflecting the dataset's nature as online retail transactions. To provide richer location information, synthetic cities are generated for each store based on the country, using locale-specific Faker instances to maintain geographic plausibility. This approach allows analysis at the store level while enhancing location details without requiring real-world addresses.*

In [6]:
# Extract unique countries as stores
store_dim = df[['Country']].drop_duplicates().reset_index(drop=True)

# Assign unique StoreID starting from 1
store_dim['StoreID'] = store_dim.index + 1

# Assign StoreName with country suffix for uniqueness
store_dim['StoreName'] = store_dim['Country'].apply(lambda x: f"Online Store - {x}")

# Assign Channel as 'Online' (dataset is online retail)
store_dim['Channel'] = 'Online'

# Generate synthetic City based on country using Faker locales
def generate_city(country):
    if country == 'United Kingdom':
        fake_local = Faker('en_GB')
    else:
        fake_local = Faker()
    return fake_local.city()

store_dim['City'] = store_dim['Country'].apply(generate_city)

store_dim.head()

,Country,StoreID,StoreName,Channel,City
0,United Kingdom,1,Online Store - United Kingdom,Online,Kimberleychester
1,France,2,Online Store - France,Online,Higginston
2,Australia,3,Online Store - Australia,Online,Deborahmouth
3,Netherlands,4,Online Store - Netherlands,Online,Berryhaven
4,Germany,5,Online Store - Germany,Online,South Lisachester


In [ ]:
fake = Faker()

# 4. Create Product Dimension Table (product_dim)
# Extract unique products from the original dataset: StockCode, Description, UnitPrice.
product_dim = df[['StockCode', 'Description', 'UnitPrice']].drop_duplicates().copy()

# Rename columns to fit dimensional model schema
product_dim = product_dim.rename(columns={
    'StockCode': 'ProductID',
    'Description': 'ProductName',
    'UnitPrice': 'UnitCost'
})

# Define a simple function to categorize products based on keywords in ProductName
def categorize_product(name):
    if pd.isna(name):
        return 'Miscellaneous'
    name = name.lower()
    if any(keyword in name for keyword in ['electronic', 'computer', 'usb', 'laptop', 'cable']):
        return 'Electronics'
    elif any(keyword in name for keyword in ['shirt', 'clothing', 'dress', 't-shirt', 'jeans']):
        return 'Clothing'
    elif any(keyword in name for keyword in ['book', 'novel', 'journal']):
        return 'Books'
    elif any(keyword in name for keyword in ['toy', 'game']):
        return 'Toys & Games'
    else:
        return 'Miscellaneous'

# Apply the category function to create a Category column
product_dim['Category'] = product_dim['ProductName'].apply(categorize_product)

# Generate a synthetic Brand name using Faker company names for each product
product_dim['Brand'] = [fake.company() for _ in range(len(product_dim))]

# Display sample of product_dim to verify
product_dim.head()

,ProductID,ProductName,UnitCost,Category,Brand
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,Miscellaneous,"Carpenter, Burton and Oneal"
1,71053,WHITE METAL LANTERN,3.39,Miscellaneous,Francis-Mann
2,84406B,CREAM CUPID HEARTS COAT HANGER,2.75,Miscellaneous,Lara-Baker
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,Miscellaneous,Diaz-Schaefer
4,84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39,Miscellaneous,Flores LLC


In [8]:
# Convert 'InvoiceDate' to datetime if not already
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Create a new column 'InvoiceDateOnly' normalized to midnight (date only, no time)
df['InvoiceDateOnly'] = df['InvoiceDate'].dt.normalize()

# Ensure 'FullDate' in time_dim is datetime type for correct merging
time_dim['FullDate'] = pd.to_datetime(time_dim['FullDate'])

### *Prepare FactSales Table (Fact Table)*

*Description:*

*This final chunk assembles the FactSales table, which records individual sales transactions linked to the various dimension tables through foreign keys. First, any encoding issues in column names are corrected to ensure consistency. Duplicate columns created through multiple merges are removed to avoid confusion and errors. The fact table is enriched by merging the Time Dimension to include a `TimeID` foreign key, facilitating time-based joins and analysis. Invoice dates are normalized to exclude time information, aligning with the date-only nature of the Time Dimension. The product identifier is standardized by assigning `ProductID` as the original stock code. The store foreign key (`StoreID`) is merged in based on country, linking sales to store locations. A key metric, `TotalSales`, is calculated by multiplying the quantity sold by the unit price, providing the total revenue per transaction line. Finally, only the relevant columns necessary for the fact table schema are selected to form the `fact_sales` DataFrame, ready for analytical queries or database loading.*


In [9]:
df = df.rename(columns={'ï»¿InvoiceNo': 'InvoiceNo'})
# Use the 'TimeID' and 'StoreID' columns without suffixes if present
if 'TimeID' not in df.columns:
    if 'TimeID_y' in df.columns:
        df['TimeID'] = df['TimeID_y']
    elif 'TimeID_x' in df.columns:
        df['TimeID'] = df['TimeID_x']

if 'StoreID' not in df.columns:
    if 'StoreID_y' in df.columns:
        df['StoreID'] = df['StoreID_y']
    elif 'StoreID_x' in df.columns:
        df['StoreID'] = df['StoreID_x']


In [10]:
df = df.loc[:,~df.columns.duplicated()]

In [11]:
# Merge df with time_dim to get TimeID by matching on normalized date columns
df = df.merge(
    time_dim[['TimeID', 'FullDate']],
    left_on='InvoiceDateOnly',
    right_on='FullDate',
    how='left'
)

# Convert 'InvoiceDate' column to just date (drop time component) for fact table compatibility
df['InvoiceDate'] = df['InvoiceDate'].dt.date

# Assign 'ProductID' as the same value as 'StockCode' for clarity and schema matching
df['ProductID'] = df['StockCode']

# Merge df with store_dim on 'Country' to get StoreID foreign key
df = df.merge(
    store_dim[['StoreID', 'Country']],
    on='Country',
    how='left'
)

# Calculate total sales amount per transaction line
df['TotalSales'] = df['Quantity'] * df['UnitPrice']

#  Add Discount column if missing (assumed 0)
if 'Discount' not in df.columns:
    df['Discount'] = 0

# Then select the columns including Discount
fact_sales = df[['InvoiceNo', 'InvoiceDate', 'TimeID', 'ProductID', 'CustomerID', 'StoreID',
                 'Quantity', 'UnitPrice', 'Discount', 'TotalSales']].copy()

## *Extract , Transform & Load:Retail_Data*

### *Imports and Logging Setup*

In [12]:
# Setup logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## *Step 1: Extraction Phase*

In [13]:
def extract_csv(csv_folder, tables):
    """
    Extract CSV files into pandas DataFrames.
    Converts InvoiceDate to datetime for FactSales.
    Returns a dictionary of {table_name: DataFrame}.
    """
    dataframes = {}
    logging.info("=== ETL PROCESS STARTED ===")
    logging.info("STEP 1: Extraction")
    
    for table in tables:
        logging.info(f"Extracting data for table '{table}' from CSV...")
        file_path = os.path.join(csv_folder, f"{table}.csv")
        try:
            df = pd.read_csv(file_path, encoding='ISO-8859-1')
            if 'InvoiceDate' in df.columns:
                df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
                df = df.dropna(subset=['InvoiceDate'])
            dataframes[table] = df
            logging.info(f"✔ Extracted {len(df)} rows from '{table}'")
        except Exception as e:
            logging.error(f"Error extracting {table}: {e}")
            dataframes[table] = pd.DataFrame()
    
    return dataframes

## *Step_2:Transformation Phase*
*Following the full extraction, we proceeded with a`* ***Full Transformation*** *approach. Each dataset was fully inspected and cleaned independently to ensure data quality and consistency before merging.*


In [14]:
def transform_data(dataframes):
    """
    Transform DataFrames:
    - FactSales: filter invalid rows, compute TotalSales, filter last year's sales
    - CustomerDim, StoreDim, TimeDim: remove duplicates, handle ID columns
    - ProductDim: remove duplicates, convert column types, remove invalid UnitPrice
    """
    logging.info("STEP 2: Transformation")
    
    for table_name, df in dataframes.items():
        if df.empty:
            logging.warning(f"Table '{table_name}' is empty, skipping transformation")
            continue

        logging.info(f"Transforming table '{table_name}'...")

        # FactSales transformations
        if table_name == "FactSales":
            if 'Quantity' in df.columns and 'UnitPrice' in df.columns:
                df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
                df.loc[:, 'TotalSales'] = round(df['Quantity'] * df['UnitPrice'], 2)
            if 'InvoiceDate' in df.columns:
                cutoff = pd.Timestamp('2024-08-12')
                df = df[df['InvoiceDate'] >= cutoff]
                df['InvoiceDate'] = df['InvoiceDate'].dt.strftime('%Y-%m-%d %H:%M:%S')

        # CustomerDim, StoreDim, TimeDim transformations
        elif table_name in ["CustomerDim", "StoreDim", "TimeDim"]:
            df = df.drop_duplicates()
            id_cols = [col for col in df.columns if "ID" in col]
            if id_cols:
                df = df.dropna(subset=id_cols)
                for col in id_cols:
                    df[col] = df[col].astype(int, errors='ignore')

        # ProductDim specific cleaning
        elif table_name == "ProductDim":
            df = df.drop_duplicates()
            for col in df.columns:
                if col.lower() in ['stockcode', 'productid']:
                    df[col] = df[col].astype(str)
                elif col.lower() in ['unitprice', 'unitcost']:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            df = df.dropna(subset=['UnitPrice']) if 'UnitPrice' in df.columns else df

        dataframes[table_name] = df
        logging.info(f"✔ Table '{table_name}' transformed. {len(df)} rows ready for loading")
    
    return dataframes

## *Step 3:Load to SQLite*

In [15]:
def load_to_db(dataframes, schema_file, db_path, csv_folder=None):
    """
    Load transformed DataFrames into SQLite database.
    Optionally, save transformed CSVs to csv_folder.
    """
    logging.info("STEP 3: Loading into database")

    # Remove existing database if it exists
    if os.path.exists(db_path):
        logging.info(f"Existing database found. Removing {db_path}")
        os.remove(db_path)

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Apply schema
    with open(schema_file, 'r') as f:
        schema_sql = f.read()
    cursor.executescript(schema_sql)
    conn.commit()
    logging.info("✔ Database schema applied successfully")

    # Insert data
    for table_name, df in dataframes.items():
        if df.empty:
            logging.warning(f"Table '{table_name}' is empty, skipping load")
            continue
        placeholders = ', '.join(['?'] * len(df.columns))
        data_tuples = list(df.itertuples(index=False, name=None))
        if table_name != "FactSales":
            query = f"INSERT OR IGNORE INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})"
        else:
            query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholders})"
        try:
            cursor.executemany(query, data_tuples)
            conn.commit()
            logging.info(f"✔ Loaded {len(df)} rows into '{table_name}'")
        except Exception as e:
            logging.error(f"Error loading {table_name}: {e}")

    conn.close()
    logging.info("=== ETL PROCESS COMPLETED SUCCESSFULLY ===")
    logging.info(f"Database created at: {db_path}")

### *Master ETL Function*

In [16]:
def etl_retail(csv_folder, schema_file, db_path):
    """
    Master ETL function that runs extraction, transformation, and loading sequentially.
    """
    tables = ["FactSales", "CustomerDim", "StoreDim", "ProductDim", "TimeDim"]

    # Step 1: Extract
    dataframes = extract_csv(csv_folder, tables)

    # Step 2: Transform
    dataframes = transform_data(dataframes)

    # Step 3: Load
    load_to_db(dataframes, schema_file, db_path)


### *Run the ETL process*

In [17]:
# Paths
csv_folder = r"C:\Users\Snit Kahsay\Desktop\DSA-2040_Practical_Exam_SnitTeshome552\Section_1\Task_2_ETL_Process_Implementation\synthetic_data"
schema_file = r"C:\Users\Snit Kahsay\Desktop\DSA-2040_Practical_Exam_SnitTeshome552\Section_1\Task_1_Data_Warehouse_Design\schema_design.sql"
db_path = r"C:\Users\Snit Kahsay\Desktop\DSA-2040_Practical_Exam_SnitTeshome552\Section_1\Task_2_ETL_Process_Implementation\retail_dw.db"

# Run ETL
etl_retail(csv_folder, schema_file, db_path)

2025-08-15 00:35:39,712 - INFO - === ETL PROCESS STARTED ===
2025-08-15 00:35:39,717 - INFO - STEP 1: Extraction
2025-08-15 00:35:39,717 - INFO - Extracting data for table 'FactSales' from CSV...
2025-08-15 00:35:41,088 - INFO - ✔ Extracted 541909 rows from 'FactSales'
2025-08-15 00:35:41,088 - INFO - Extracting data for table 'CustomerDim' from CSV...
2025-08-15 00:35:41,144 - INFO - ✔ Extracted 4389 rows from 'CustomerDim'
2025-08-15 00:35:41,146 - INFO - Extracting data for table 'StoreDim' from CSV...
2025-08-15 00:35:41,150 - INFO - ✔ Extracted 38 rows from 'StoreDim'
2025-08-15 00:35:41,158 - INFO - Extracting data for table 'ProductDim' from CSV...
2025-08-15 00:35:41,245 - INFO - ✔ Extracted 18053 rows from 'ProductDim'
2025-08-15 00:35:41,249 - INFO - Extracting data for table 'TimeDim' from CSV...
2025-08-15 00:35:41,258 - INFO - ✔ Extracted 305 rows from 'TimeDim'
2025-08-15 00:35:41,261 - INFO - STEP 2: Transformation
2025-08-15 00:35:41,263 - INFO - Transforming table 'Fac

### *Checking the loading the data*

In [ ]:

db_path = r"C:\Users\Snit Kahsay\Desktop\DSA-2040_Practical_Exam_SnitTeshome552\Section_1\Task_2_ETL_Process_Implementation\retail_dw.db"

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in DB:", tables)

# Query more rows (e.g., 20 rows)
cursor.execute("SELECT * FROM FactSales LIMIT 20;")
rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()

Tables in DB: [('TimeDim',), ('CustomerDim',), ('ProductDim',), ('StoreDim',), ('FactSales',), ('sqlite_sequence',)]
(1, '536365', '2024-12-01 00:00:00', 20241201, '85123A', 17850, 1, 6, 2.55, 0.0, 15.3)
(2, '536365', '2024-12-01 00:00:00', 20241201, 71053, 17850, 1, 6, 3.39, 0.0, 20.34)
(3, '536365', '2024-12-01 00:00:00', 20241201, '84406B', 17850, 1, 8, 2.75, 0.0, 22.0)
(4, '536365', '2024-12-01 00:00:00', 20241201, '84029G', 17850, 1, 6, 3.39, 0.0, 20.34)
(5, '536365', '2024-12-01 00:00:00', 20241201, '84029E', 17850, 1, 6, 3.39, 0.0, 20.34)
(6, '536365', '2024-12-01 00:00:00', 20241201, 22752, 17850, 1, 2, 7.65, 0.0, 15.3)
(7, '536365', '2024-12-01 00:00:00', 20241201, 21730, 17850, 1, 6, 4.25, 0.0, 25.5)
(8, '536366', '2024-12-01 00:00:00', 20241201, 22633, 17850, 1, 6, 1.85, 0.0, 11.1)
(9, '536366', '2024-12-01 00:00:00', 20241201, 22632, 17850, 1, 6, 1.85, 0.0, 11.1)
(10, '536367', '2024-12-01 00:00:00', 20241201, 84879, 13047, 1, 32, 1.69, 0.0, 54.08)
(11, '536367', '2024-12-0